# **Web scraping application with Python using Selenium**

### **Extraction of historical prices of IBEX35 indices from a website**

### **Consideraciones legales y éticas**

Esta publicación no trata sobre cómo extraer datos de una página web con fines ilegales.

Hay que asegurarse de tener permiso antes de extraer ciertos tipos de datos que puede violar los términos del servicio o incluso regulaciones legales:

- Revise los términos de uso de la página web en relación a los permisos de extracción de datos.
- Priorice el uso de las APIs, si están disponibles, ya que proporcionan acceso legal a los datos.
- Póngase en contacto directamente con el propietario de la página web para comprobar el permiso de extracción de datos.

### **Introducción**

En este proyecto veremos cómo se pueden obtener datos de una tabla de precios históricos de los índices IBEX. Para ello deberemos introducir en un **formulario** tres campos:

- `Desde`: Fecha de comienzo del histórico. Como mínimo podemos introducir la fecha correspondiente a un año anterior a la fecha actual.
- `Hasta`: Fecha de finalización del histórico. Como máximo podemos introducir el día anterior a la fecha actual.
- `Indice`: Es una lista desplegable de índices. Debemos seleccionar uno.

<img src = './img/web_principal_ibex_formulario.jpg' width =800)>

Finalmente, pulsaremos sobre `BUSCAR` para enviar el formulario y que nos devuelva la tabla de resultados que queremos obtener.

Por último extraeremos los datos de dicha tabla y la guardaremos en un csv.

**Categorías de los índices**

Vemos algunos de los índices que podemos aplicar

Índices Generales:
- IBEX 35®
- IBEX 35® con Dividendos
- IBEX MEDIUM CAP®
- IBEX SMALL CAP®

Índices por Sector:
- IBEX 35® Bancos
- IBEX 35® Energía
- IBEX 35® Construcción

Índices de Igualdad de Género:
- IBEX Gender Equality
- IBEX Gender Equality Total Return
- IBEX Gender Equality Net Return

Índices Apalancados e Inversos:
- IBEX 35® Inverso
- IBEX 35® Doble Apalancado
- Índice TEF Apalancado X3

Índices de ESG (Medioambiental, Social y de Gobernanza):
- IBEX ESG
- IBEX ESG Weighted
- FTSE4Good IBEX

### **Conocimientos previos sobre Selenium**

Selenium es una herramienta de automatización web que puede interactuar con páginas web, lo que nos permite extraer contenido cargado dinámicamente por JavaScript, como los sitios construidos con React o Angular. Puede simular un usuario real navegando por la web, haciendo clic en botones y completando formularios.

En este caso, al tener que obtener datos de una tabla dinámica, que además se obtiene cubriendo un formulario que se envía a través de javascript, Selenium es la mejor opción.

**Localizar elementos de un HTML**

Selenium ofrece dos técnicas para localizar elementos HTML en páginas web para realizar un web scraping: `**.find_element**` y  `**.find_elements**`. El método  find_element busca un único elemento específico en la página web, mientras que el find_elements recupera una lista que contiene todos los elementos descubiertos en la página web.

Para localizar elementos en la página con estos métodos podemos usar la clase By con los siguientes atributos:
- By.ID = "id" --> Localiza elementos en función de su "id".
- By.NAME = "name" --> Localiza elementos en función de su atributo de "nombre".
- By.XPATH = "xpath" --> Localiza elementos basándose en una expresión XPath. Se puede usar cuando no tengamos el atributo "id" o "name" adecuado.
- By.LINK_TEXT = "link text" # Se puede usar cuando conozcamos el texto del enlace usado dentro de una etiqueta `<a>`.
- By.PARTIAL_LINK_TEXT = "partial link text" # Similar al anterior.
- By.TAG_NAME = "tag name" --> Localiza elementos según su nombre de etiqueta.
- By.CLASS_NAME = "class name" --> Localiza elementos según el nombre de la clase.
- By.CSS_SELECTOR = "css selector" --> Localiza elementos según el selector CSS.

[Aquí](https://selenium-python.readthedocs.io/locating-elements.html) podemos encontrar más información sobre los diferentes métodos que hay para localizar elementos en una página.

### **Instalación de dependencias**

A continuación se muestran las librerías necesarias para la ejecución del código creado:

- ipykernel : kernel de Jupyter necesario para ejecutar Python.
- tqdm : herramienta útil para crear barras de progeso en bucles.
- joblib : biblioteca que permite la serialización de objetos Python.
- pandas : biblioteca escrita sobre Numpy para la manipulación y el análisis de datos.

Ejecutamos en la terminal el siguiente código:

`pip install ipykernel tqdm joblib pandas`

### **Instalación de selenium**

Podemos instalar el selenium de 2 maneras:

1. Manualmente:

- Descargarlo en https://pypi.org/project/selenium/#files y descomprimirlo
- Ejecutar: python setup.py install

2. Mediante pip (package installer for python):

- Ejecutar: pip install -U selenium

Utilizando la segunda opción, ejecuto en el terminal:

`pip install -U selenium`

### **Instalación del driver del navegador**

Necesitamos un driver para que selenium haga interfaz con el navegador seleccionado. Algunos de ellos:

- Safari -> https://webkit.org/blog/6900/webdriver-support-in-safari-10/
- Firefox/mozilla -> https://github.com/mozilla/geckodriver/releases
- Edge -> https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
- Chrome -> https://chromedriver.chromium.org/downloads

Por otro lado, "webdriver_manager" es una biblioteca que permite gestionar automáticamente los controladores de diferentes navegadores. En la actualidad soporta los siguientes controladores:

- ChromeDriver
- EdgeChromiumDriver
- GeckoDriver
- IEDriver
- OperaDriver

En mi caso, haré uso de Chrome, por lo que instalaré el "webdriver_manager" vía "pip":

`pip install webdriver_manager`

### **Importación de dependencias**

In [1]:
import pandas as pd
import time
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

### **Configuración del navegador**

Configuramos las opciones de comienzo del navegador para evitar que se muestre la pantalla de bienvenida y establezco la ruta de un perfil que tengo predefinido para usar con el navegador Chrome.

In [2]:
# Set up Chrome options
chrome_options = Options()
#chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--no-first-run") # Desactiva la pantalla de bienvenida
chrome_options.add_argument("--no-default-browser-check") # Desactiva la comprobación que hace chrome por defecto al iniciarse
chrome_options.add_argument("user-data-dir=C:/Users/jagui/AppData/Local/Google/Chrome/User Data/Profile 1") # ruta de un perfil preestablecido
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
chrome_options.add_argument("--no-sandbox")  # Required for some environments
chrome_options.add_argument("--window-size=1920x1080")  # Set window size to ensure visibility

### **Abrir el navegador**

In [3]:
# Configurar el service con ChromeDriverManager
service = Service(ChromeDriverManager().install())

# Crear una nueva instancia del webDriver de Chrome con las opciones configuradas 
driver = webdriver.Chrome(service=service, options=chrome_options)

# Abrir la página web
url = "https://www.bolsasymercados.es/bme-exchange/es/Indices/Ibex/Precios-Historicos"
driver.get(url)

# Aceptación de cookies
try:
    driver.find_element(By.ID, 'onetrust-accept-btn-handler').click()    
    print("Las cookies han sido aceptadas")
    
except:
    pass
    
print("Las cookies ya se han aceptado anteriormente")

# Esperamos un tiempo hasta que el form se cargue
wait = WebDriverWait(driver, 5)

# Esperar un breve momento para evitar problemas con la carga de la página
#time.sleep(5)

Las cookies ya se han aceptado anteriormente


### **Inspeccionar la página web**

Al analizar la web vemos que sólo nos aparece el formulario, que tenedremos que cubrir para obtener la tabla de datos deseada.

**Formulario**

Podemos observar que se encuentra dentro de `<form>` y dentro tiene un `<div>` para los campos "Desde", "Hasta" e "Índice" y un `<input>` para "BUSCAR".

Dentro del primer `<div>` hay otros `<div>` con clase "form-group", uno para cada uno de los campos nombrados anteriormente.

<img src = './img/ibex_formulario.jpg'>

Entramos en los dos primeros "form-group" que contienen  los campos "Desde" y "Hasta". En ellos nos encontramos con unos `<input>` que se usarán para introducir las fechas en dichos campos. 

También podemos observar que nos indican las fechas mínimas y máximas que podemos introducir y la fecha por defecto que aparece en el formulario.

<img src = './img/ibex_formulario_desde_hasta.jpg'>

Dentro del segundo `<div>` nos encontramos con un selector en el cual aparece un listado de todos los índices que disponemos para obtener la tabla.

<img src = './img/ibex_formulario_indice.jpg'>

**Tabla de datos**

Hacemos, por ejemplo, una búsqueda con las fechas e Índice que vienen por defecto y pulsamos el botón "Buscar". Nos devuelve la siguiente tabla:

<img src = './img/ibex_tabla.jpg'>

Analizando el HTML, vemos que la tabla tiene dos partes, un "head" y un "Body". 

Dentro del `<thead>` nos encontramos un `<tr>` que es la única fila que tiene la cabecera. Dentro del `<tr>`nos encontramos con 6 `<th>` que es cada uno de los campos de la cabecera.

Dentro del `<tbody>`nos encontramos con tantos `<tr>`como filas se hayan creado en la tabla. Dentro de cada uno de los `<tr>`nos encontramos con 6 `<th>` que nos devuelven los valores de los 6 campos de la cabecera.

<img src = './img/ibex_tabla_head_body.jpg'>

### **Extracción de datos**

**Listado de índices**

Genero un diccionario con los índices que aparecen en el desplegable, de esa forma tengo guardados los códigos que se el javascript utiliza para enviar el formulario.

In [4]:
# diccionario con la lista desplegable de los índices
index_sector = driver.find_element(By.ID, 'indexSector')
options_sector = index_sector.find_elements(By.TAG_NAME, 'option')

dic_index = {}
for option in options_sector:
    key = option.get_attribute('value')
    value = option.text.strip() # elimina espacios en blanco iniciales y finales
    dic_index[key] = value 

dic_index

{'ES0SI0000005': 'IBEX 35®',
 'ES0SI0000047': 'IBEX 35® con Dividendos',
 'ES0SI0000013': 'IBEX MEDIUM CAP®',
 'ES0SI0000021': 'IBEX SMALL CAP®',
 'ES0S00000901': 'IBEX 35® Bancos',
 'ES0S00000919': 'IBEX 35® Energía',
 'ES0S00000927': 'IBEX 35® Construcción',
 'ES0S00001586': 'IBEX Gender Equality',
 'ES0S00001594': 'IBEX Gender Equality Total Return',
 'ES0S00001602': 'IBEX Gender Equality Net Return',
 'ES0SI0000039': 'IBEX TOP Dividendo®',
 'ES0SI0000062': 'IBEX 35® con Dividendos Netos',
 'ES0SI0000054': 'IBEX 35® Inverso',
 'ES0SI0000070': 'IBEX 35® Doble Inverso',
 'ES0SI0000088': 'IBEX 35® Inverso X3',
 'ES0SI0000195': 'IBEX 35® Inverso X5',
 'ES0SI0001730': 'IBEX 35® Inverso X10',
 'ES0SI0000096': 'IBEX 35® Doble Apalancado',
 'ES0SI0000112': 'IBEX 35® Doble Apalancado Bruto',
 'ES0SI0000138': 'IBEX 35® Doble Apalancado Neto',
 'ES0SI0000104': 'IBEX 35® Apalancado X3',
 'ES0SI0000179': 'IBEX 35® Apalancado Neto X3',
 'ES0SI0000187': 'IBEX 35® Apalancado Neto X5',
 'ES0SI000172

**Cubrir el formulario**

Definimos los campos que queremos introducir. Los campos de las fechas disponen de un calendario dinámico para introducirlas, por lo que tenemos que ir poniendo manualmente el día, el mes y el año en cada uno de los campos. Y no podemos introducirlos en ese orden, sino que debemos hacerlo en el orden en que está configurada la página.

In [5]:
# Indice del sector
key_sector = "ES0SI0000005" # Seleccionar la opción deseada. En este caso 'IBEX 35®'

# Fecha inicio (2 opciones --> fecha mínima disponible o fecha deseada)
start_date_str = (datetime.now() - timedelta(days=366)).strftime('%Y-%m-%d')
#start_date_str = "2024-03-21" # Poner la fecha deseada

# Fecha fin (2 opciones --> fecha máxima disponible o fecha deseada)
end_date_str = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d') # fecha máxima disponible
# end_date_str = "2024-08-20" # Poner la fecha desada

# día, mes y año del start_date
start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
start_day = start_date.day
start_month = start_date.month
start_year = start_date.year

# día, mes y año del end_date
end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
end_day = end_date.day
end_month = end_date.month
end_year = end_date.year

Enviamos al formulario los campos que acabamos de definir

In [6]:
# Enviar el start date
start_date_element = wait.until(EC.presence_of_element_located((By.ID, 'date-from')))
start_date_element.clear()
start_date_element.send_keys(Keys.TAB)
start_date_element.send_keys(Keys.TAB)
start_date_element.click()
start_date_element.send_keys(f"{start_year}") 
time.sleep(0.5) 
start_date_element.send_keys(Keys.ARROW_LEFT)
start_date_element.send_keys(f"{start_month}")
time.sleep(0.5)
start_date_element.send_keys(Keys.ARROW_LEFT)
start_date_element.send_keys(Keys.ARROW_LEFT)
start_date_element.send_keys(f"{start_day}")
time.sleep(0.5)

In [7]:
# Enviar el end date
end_date_element = wait.until(EC.presence_of_element_located((By.ID, 'date-to')))
end_date_element.clear()
end_date_element.click()
end_date_element.send_keys(Keys.TAB)
end_date_element.send_keys(Keys.TAB)
end_date_element.send_keys(f"{end_year}") 
time.sleep(0.5) 
end_date_element.send_keys(Keys.ARROW_LEFT)
end_date_element.send_keys(f"{end_month}")
time.sleep(0.5)
end_date_element.send_keys(Keys.ARROW_LEFT)
end_date_element.send_keys(Keys.ARROW_LEFT)
end_date_element.send_keys(f"{end_day}")
time.sleep(0.5)

In [8]:
# Seleccionar la opción deseada del menú desplegable usando la clase Select
dropdown_element = Select(driver.find_element(By.ID, 'indexSector'))
dropdown_element.select_by_value(key_sector)  

In [9]:
# Enviar el formulario
submit_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Buscar']")
submit_button.click()

In [10]:
# Espera a que los resultados se carguen
wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))  # Ajustar si los resultados de la tabla todavía no han aparecido

<selenium.webdriver.remote.webelement.WebElement (session="233899825279d866caf318c6b643e147", element="f.70197DB557E61B61C811BC6F976B5CDB.d.0531544AB0813521899EF8DB4D846356.e.193")>

**Extracción de datos de la tabla**

Dependiendo de las fechas que introduzcamos, la tabla que nos devuelve puede estar en una sóla página o en varias. En nuestro caso vemos que se encuentra en varias páginas, por lo que para poder pasar de una página a otra tenemos que interactuar con ella.

<img src ='./img/ibex_navegacion_paginas.jpg'>

Para hacer la extracción de los datos de la tabla, definimos una función que usaremos para los datos de la primera página.

In [11]:
# Función para extraer datos de la tabla en la primera página
def extract_table_data():
    table = driver.find_element(By.TAG_NAME, 'table')
    rows = table.find_elements(By.TAG_NAME, 'tr')
    header = []
    page_data = []
    headers = rows[0].find_elements(By.TAG_NAME, 'th')
    header.extend([head.text for head in headers])
    for row in rows[1:]:
        cols = row.find_elements(By.TAG_NAME, 'td')
        page_data.append([col.text for col in cols])    

    return header, page_data

In [12]:
# Inicializar una lista para almacenar los datos de todas las páginas
all_data = []

# extraemos los datos de la primera página
header, page_data = extract_table_data()
all_data.extend(page_data)

Navegamos por el resto de las páginas y vamos extrayendo los datos a medida que pasamos por ellas y los vamos almacenando.

In [13]:
# Navegar por las páginas y extraer datos
while True:
    try:
        # Encontrar el botón de siguiente página y hacer clic en él
        next_page = driver.find_element(By.XPATH, "//div[@class='col-sm-12']//a[@href='#' and @aria-disabled='false']//span[@class='glyphicon glyphicon-forward']")  # Cambia el XPATH si es necesario
        next_page.click()

        # Esperar a que la nueva página se cargue
        wait.until(EC.presence_of_element_located((By.TAG_NAME, 'table')))
        
        # Esperar un breve momento para evitar problemas con la carga de la página
        time.sleep(3)
        
        # Extraer datos de la página actual
        header, page_data = extract_table_data()
        all_data.extend(page_data)             
            
    except Exception as e:
        print("No hay más páginas o se produjo un error:", e)
        break

No hay más páginas o se produjo un error: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='col-sm-12']//a[@href='#' and @aria-disabled='false']//span[@class='glyphicon glyphicon-forward']"}
  (Session info: chrome=128.0.6613.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00828213+26163]
	(No symbol) [0x007B9CC4]
	(No symbol) [0x006B24C3]
	(No symbol) [0x006F7453]
	(No symbol) [0x006F762B]
	(No symbol) [0x00736B62]
	(No symbol) [0x0071AD04]
	(No symbol) [0x00734661]
	(No symbol) [0x0071AA56]
	(No symbol) [0x006EBE89]
	(No symbol) [0x006EC8CD]
	GetHandleVerifier [0x00AFD313+2996019]
	GetHandleVerifier [0x00B51B89+3342249]
	GetHandleVerifier [0x008B7AEF+614159]
	GetHandleVerifier [0x008BF17C+644508]
	(No symbol) [0x007C27FD]
	(No symbol) [0x007BF6F8]
	(No symbol) [0x007BF895]
	(No symbol) [0x007B1C16]
	Base

### **Guardar los datos en un archivo**

Cargamos en un DataFrame todos los datos que hemos almacenado en "all_data" y posteriormente lo guardamos en un archivo csv.

In [14]:
# Convertir los datos en un DataFrame
df = pd.DataFrame(all_data, 
                  columns = header
                  )
df

,Fecha,Último,Anterior,Máximo,Mínimo,Medio
0,08/09/2023,"9.364,60","9.310,00","9.372,30","9.238,50","9.304,80"
1,11/09/2023,"9.435,20","9.364,60","9.457,90","9.367,30","9.416,00"
2,12/09/2023,"9.455,40","9.435,20","9.501,70","9.448,00","9.470,70"
3,13/09/2023,"9.424,10","9.455,40","9.448,30","9.330,60","9.389,20"
4,14/09/2023,"9.549,00","9.424,10","9.563,10","9.380,00","9.469,20"
...,...,...,...,...,...,...
250,02/09/2024,"11.395,30","11.401,90","11.422,40","11.337,50","11.386,90"
251,03/09/2024,"11.279,20","11.395,30","11.429,40","11.257,50","11.319,10"
252,04/09/2024,"11.213,90","11.279,20","11.238,30","11.138,80","11.201,30"
253,05/09/2024,"11.273,50","11.213,90","11.317,30","11.152,20","11.272,50"


In [15]:
# guardar en un archivo csv
df.to_csv(dic_index[key_sector] + '_historico.csv')

In [16]:
# Cerramos el navegador
driver.quit()

### **Conclusiones**

El web scraping con Selenium es una herramienta poderosa y flexible para extraer datos de sitios web dinámicos, especialmente aquellos que dependen en gran medida de JavaScript para generar contenido. A lo largo de este artículo, hemos visto cómo Selenium se destaca en situaciones donde otras herramientas de scraping, como BeautifulSoup o Scrapy, pueden tener limitaciones debido a la necesidad de interactuar con elementos de la página o manejar eventos complejos.

Sin embargo, aunque Selenium ofrece grandes ventajas, también presenta ciertas limitaciones y desventajas frente a otras herramientas. Por ejemplo:

- Rendimiento y eficiencia : Selenium ejecuta un navegador completo, lo que puede ser más lento y consumir más recursos. Esto puede ser una limitación si se necesita extraer grandes cantidades de datos de forma rápida y eficiente.
- Mantenimiento de scripts: Dado que Selenium interactúa con la interfaz visual de las páginas web, los scripts de scraping pueden volverse frágiles ante cambios en el diseño o la estructura del DOM de la página. 

En este artículo nos hemos encontrado con un escenario complejo, en el que para acceder a los datos que queríamos, primero tuvimos que cubrir un formulario, que la página envía a través de javascript, para finalmente obtener la tabla de datos específica. Por ello, la utilización de Selenium es la mejor opción para este tipo de casos.
Finalmente hemos podido extraer el histórico de un año de los precios del índice IBEX35.

### **Referencias**

- [Python](https://www.python.org/)
- [Ipykernel](https://pypi.org/project/ipykernel/)
- [tqdm](https://pypi.org/project/tqdm/)
- [joblib](https://pypi.org/project/joblib/)
- [Pandas](https://pandas.pydata.org/)
- [Selenium](https://www.selenium.dev/)
- [Locating Elements with selenium](https://selenium-python.readthedocs.io/locating-elements.html)
- [webdriver-manager](https://pypi.org/project/webdriver-manager/)